# Cells
A normal cell is built by:
1. Receiving two input values (the output of previous cell and a residue(skip) from two cells earlier)
2. 6 outputs from 5 nodes. The input of a node are two values from previous outputs and the two original inputs of the cell. The output of the node is the sum of the result of aplying an operation to each of the inputs.The final output of the cell is the concatenation of the last two unused outputs. 
3. Allowed operations: Identity, Separable convolution with filter size: 3x3, 5x5 and 7x7, 3x3 Avg pooling,  3x3 max pooling, 3x3 Separable dilated convolution. 

A Reduction cell has the same structure as a normal cell except that the final output is passed through an avg pooling of stride 2x2

# Architecture of an AmoebaNet
The full architecture of the net is given simply by the structure of normal and reduction cells. After these are fixed, the net consists of: 

Input Layer
Conv 3x3, filters = 24, stride = 2
2 Reduction cells
3 Normal cells
Reduction cell
3 Normal cells
Reduction cell
3 Normal cells
Flatten (or MaxPooling)
Softmax, outputs = 10 (CIFAR-10)


# Operations
0. Identity
1. SepConv 3x3
2. SepConv 5x5
3. SepConv 7x7
4. AvgPool 3x3
5. MaxPool 3x3
6. SepDilConv 3x3

# Genome
A genome corresponds to the codification of the structure of the normal and reduction cells. Since the only difference between them is the size reduction in the latter, we encode both type of cells the same way. 

Encoding of a cell: Node1+Node2+Node3+Node4+Node5
    Node1: 8 bits, 3 for Operation1, 3 for Operation2, 1 for which operation is applied to input0, and 1 for which operation is applied to input1
    e.g. (001,010)+ (1,0): apply SepConv3x3 to input1, apply SepConv5x5 to input0. Output = sum of results
    Node2: 12 bits, 6 for operations, 2 for the application of ops to inputs, 2 for choosing input from the previous 3 inputs (the two original inputs and the output of Node1) 
    Node3: 12 bits: same as Node2 but the last four bits are to choose inputs from 0,1,2,3
    Node4: 14 bits: same as Node3but the last 6 bits are to choose inputs from 0,1,2,3,4
    Node5: 14 bits: same as Node4
Length of encoding of a cell: 60 bits
 
GENOME = code of Normal cells + code of Reduction Cells, i.e.
        = code of a cell + code of a cell
        
Length of genome = 120 bits

# An architecture consists of the settings for normal and reduction cells
arch = settingsNormalCell + settingsReductionCell

SettingsCell: (integers)

Node1 = (Op1, Op2) + (Bin1, Bin2)  Two integers 0--7, 2 binaries

Node2 = (Op1, Op2) + (Bin1, Bin2) + (Input1, Input2) Two integers 0--7, 2 binaries, 2 integers 0--2

Node3 = (Op1, Op2) + (Bin1, Bin2) + (Input1, Input2) Two integers 0--7, 2 binaries, 2 integers 0--3

Node4 = (Op1, Op2) + (Bin1, Bin2) + (Input1, Input2) Two integers 0--7, 2 binaries, 2 integers 0--4

Node5 = (Op1, Op2) + (Bin1, Bin2) + (Input1, Input2) Two integers 0--7, 2 binaries, 2 integers 0--4

Total: 28 integers

Example: 
cell = 3210 240121 241130 240120 241043
arch = 3210 240121 241130 240120 241043 + 3210 240121 241130 240120 241043

In [31]:
def bin1(num):
    return bin(num)[2:]
    
def bin2(num):
    st = bin(num)[2:]
    if len(st) == 2:
        return st
    else:
        return '0' + st
    
def bin3(num):
    st = bin(num)[2:]
    while len(st) < 3:
        st = '0' + st
    return st

In [34]:
def cell_to_binary(cell):
    # cell an array of 28 integers
    enconding = ''
    #Node 1
    encoding =  bin3(cell[0]) + bin3(cell[1]) + bin1(cell[2]) + bin1(cell[3])
    #Node 2
    if 2 < cell[8]: cell[8] = random.randint(0,2)
    if 2 < cell[9]: cell[9] = random.randint(0,2)
    encoding += bin3(cell[4]) + bin3(cell[5]) + bin1(cell[6]) + bin1(cell[7]) + bin2(cell[8]) + bin2(cell[9])
    #Node 3
    encoding += bin3(cell[10]) + bin3(cell[11]) + bin1(cell[12]) + bin1(cell[13]) + bin2(cell[14]) + bin2(cell[15])
    #Node 4
    if 4 < cell[20]: cell[20] = random.randint(0,4)
    if 4 < cell[21]: cell[21] = random.randint(0,4)
    encoding +=  bin3(cell[16]) + bin3(cell[17]) + bin1(cell[18]) + bin1(cell[19]) + bin3(cell[20]) + bin3(cell[21])
    #Node 5
    if 4 < cell[26]: cell[26] = random.randint(0,4)
    if 4 < cell[27]: cell[27] = random.randint(0,4)
    encoding +=  bin3(cell[22]) + bin3(cell[23]) + bin1(cell[24]) + bin1(cell[25]) + bin3(cell[26]) + bin3(cell[27])
    
    return encoding

In [36]:
arr =list(map(int, '3210240121241130240120241043'))
code = cell_to_binary(arr)
print(code, len(code))

011010100101000110010101001111000101000101000001010010100011 60
